# Análise



## Introdução



### Objetivos da Análise

- Visualisar variáveis com maior impacto na demanda e na receita
- Visualizar dias e horários com maior demanda



### Sobre o Dataset

Este dataset contém dados de 1000 pedidos de um restaurante.

## Preparação dos dados

### Carregamento de dados

In [ ]:
# Importando a biblioteca Pandas
import pandas as pd

In [ ]:
# Carregando dataset
df = pd.read_csv('food_sales.csv')

In [ ]:
# Exibição prévia dos dados
df.head()

### Limpeza de dados

In [ ]:
# Informações sobre o dataset
df.info()

**Observações iniciais**:
- Há valores nulos na coluna `transaction_type`, representando mais de 10% dos dados.
- As colunas categóricas estão no formato `object`.
- A coluna `date` possui dados inconsistentes em relação ao formato de data.
- É importante para a análise extrair ano, mês e dia da semana da coluna `date`.

In [ ]:
# Substituindo valores nulos pela categoria Unknow
df['transaction_type'] = df['transaction_type'].fillna('Unknow')

In [ ]:
# Ajustando variáveis categóricas
category_columns = ['item_name', 'item_type', 'transaction_type', 'received_by', 'time_of_sale']
df[category_columns] = df[category_columns].astype('category')

In [ ]:
# Ajustando a variável date para datetime
df['date'] = df['date'].str.replace('/', '-', regex=False)
df['date'] = pd.to_datetime(df['date'], format='%m-%d-%Y')

In [ ]:
# Extraindo ano, mês e dia da semana de date
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month_name()
df['weekday'] = df['date'].dt.day_name()

In [ ]:
# resultado final
df.head()

## Análise de produtos

### Desempenho por categoria

Resumo da quantidade vendida e da receita por categoria.

In [ ]:
# Quantidade e receita por categoria
df.groupby('item_type', observed=True)[['quantity', 'transaction_amount']].sum()

In [ ]:
teste = df.groupby('item_type', observed=True)[['quantity', 'transaction_amount']].sum()
teste.plot(
    title=['Quantidade', 'Receita'],
    kind='pie',
    subplots=True,
    autopct='%1.1f%%',
    startangle=90,
    ylabel='',
    legend=False,
    #layout=(1,2)
);

Conclui-se que **Fastfood** é a categoria com maior impacto na quantidade e na receita.

### Desempenho por item e receita

Resumo da quantidade vendida e da receita por item.

In [ ]:
# Quantidade e receita por item
df_item_amount = df.groupby('item_name', observed=True)[['quantity', 'transaction_amount']].sum()
df_item_amount

In [ ]:
df_item_amount.sort_values('item_name', ascending=False).plot(
    y=['quantity', 'transaction_amount'],
    title=['Quantidade', 'Receita'],
    kind='barh',
    subplots=True,
    ylabel='',
    legend=False,
    sharey=False,
    sharex=False,
    figsize=(6,7.4)
);

Conclui-se que **Cold coffee** tem maior impacto na quantidade vendida, enquanto **Sandwich** tem maior impacno na receita.

## Análise de vendas

### Desempenho por vendedor

In [ ]:
order_received = df.groupby('received_by', observed=True)
graph = order_received.agg(
    {
        'received_by': 'count',
        'transaction_amount': 'sum'
    }
)

In [ ]:
graph

In [ ]:
graph.plot(
    title=['Pedidos', 'Receita'],
    kind='pie',
    subplots=True,
    autopct='%1.1f%%',
    startangle=90,
    ylabel='',
    legend=False,
);

## Análise temporal

### Desempenho anual

In [ ]:
desempenho = df.groupby('year', observed=True)

desempenho_agg = desempenho.agg(
    {
        'quantity': 'sum',
        'year': 'count',
        'transaction_amount': 'sum'
    }
)

In [ ]:
desempenho_agg

In [ ]:
desempenho_agg.plot(
    title=['Quantidade', 'Pedidos', 'Receita'],
    kind='bar',
    subplots=True,
    ylabel='',
    xlabel='',
    legend=False,
    layout=(2, 2),
    figsize=(9,8),
    sharex=False,
    rot=0,
);

### Desempenho mensal

In [ ]:
df_22 = df[df['year'] == 2022]
df_23 = df[df['year'] == 2023]

#### Itens por mês

2022

In [ ]:
# Itens 2022
items_month = df_22.groupby('month', observed=True)['quantity'].sum()
items_month.plot(kind='barh');

2023

In [ ]:
# Itens 2023
items_month = df_23.groupby('month', observed=True)['quantity'].sum()
items_month.plot(kind='barh');